# Day 2: Chunking

Today we are going to explore the types of chunking.

In [2]:
#Let's add the markdown download code here
import io
import zipfile
import requests
import frontmatter

def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown files from a GitHub repository.

    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name

    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com'
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)

    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))

    for file_info in zf.infolist():
        filename = file_info.filename
        filename_lower = filename.lower()

        if not (filename_lower.endswith('.md')
            or filename_lower.endswith('.mdx')):
            continue

        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                post = frontmatter.loads(content)
                data = post.to_dict()
                data['filename'] = filename
                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

    zf.close()
    return repository_data

In [3]:
print("Hello from course!")
balsam_faq = read_repo_data('Zesky665', 'balsam')
evidently_docs = read_repo_data('evidentlyai', 'docs')
    
print(f"FAQ documents: {len(balsam_faq)}")
print(f"Evidently documents: {len(evidently_docs)}")
    

Hello from course!
FAQ documents: 3
Evidently documents: 95


In [ ]:
# Now we can start chunking
# There are multiple ways of chunking a document.
# Here they are in order of complexity:
# - Simple Chunking
# - Token Based Chunking
# - Sematinc Chunking
# - Paragrapgh Splitting
# - Section Splitting
# - AI-powered Splitting

In [4]:
# The most commonly used simple chunking method is sliding window, which is chunking with overlap.
def sliding_window(seq, size, step):
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    n = len(seq)
    result = []
    for i in range(0, n, step):
        chunk = seq[i:i+size]
        result.append({'start': i, 'chunk': chunk})
        if i + size >= n:
            break

    return result

In [5]:
evidently_chunks = []

for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy['content']
    chunks = sliding_window(doc_content, 2000, 1000)
    for chunk in chunks:
        chunk.update(doc_copy)
    evidently_chunks.extend(chunks)

In [7]:
# Section Splitting
import re
text = evidently_docs[45]['content']
paragraphs = re.split(r"\n\s*\n", text.strip())

In [9]:
import re

def split_markdown_by_level(text, level=2):
    """
    Split markdown text by a specific header level.

    :param text: Markdown text as a string
    :param level: Header level to split on
    :return: List of sections as strings
    """
    # This regex matches markdown headers
    # For level 2, it matches lines starting with "## "
    header_pattern = r'^(#{' + str(level) + r'} )(.+)$'
    pattern = re.compile(header_pattern, re.MULTILINE)

    # Split and keep the headers
    parts = pattern.split(text)

    sections = []
    for i in range(1, len(parts), 3):
        # We step by 3 because regex.split() with
        # capturing groups returns:
        # [before_match, group1, group2, after_match, ...]
        # here group1 is "## ", group2 is the header text
        header = parts[i] + parts[i+1]  # "## " + "Title"
        header = header.strip()

        # Get the content after this header
        content = ""
        if i+2 < len(parts):
            content = parts[i+2].strip()

        if content:
            section = f'{header}\n\n{content}'
        else:
            section = header
        sections.append(section)

    return sections

In [10]:
evidently_chunks = []

for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    sections = split_markdown_by_level(doc_content, level=2)
    for section in sections:
        section_doc = doc_copy.copy()
        section_doc['section'] = section
        evidently_chunks.append(section_doc)

In [ ]:
# LLM Based Chunking
import os
from mistralai import Mistral

api_key=os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

def llm(prompt, model="mistral-large-latest"):
    chat_response = client.chat.complete(
        model= model,
        messages = [
            {
                "role": "user",
                "content": "What is the best French cheese?",
            },
        ]
    )
    print(chat_response.choices[0].message.content)
    return chat_response.choices[0].message.content



In [17]:
prompt_template = """
Split the provided document into logical sections
that make sense for a Q&A system.

Each section should be self-contained and cover
a specific topic or concept.

<DOCUMENT>
{document}
</DOCUMENT>

Use this format:

## Section Name

Section content with all relevant details

---

## Another Section Name

Another section content

---
""".strip()

In [ ]:
def intelligent_chunking(text):
    prompt = prompt_template.format(document=text)
    response = llm(prompt_template)
    sections = response.split('---')
    sections = [s.strip() for s in sections if s.strip()]
    return sections

In [ ]:
from tqdm.auto import tqdm

evidently_chunks = []

for doc in tqdm(evidently_docs):
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')

    sections = intelligent_chunking(doc_content)
    for section in sections:
        section_doc = doc_copy.copy()
        section_doc['section'] = section
        evidently_chunks.append(section_doc)

  0%|          | 0/95 [00:00<?, ?it/s]

The "best" French cheese is highly subjective—it depends on personal taste, texture preferences, and how you plan to enjoy it (on a cheese board, melted, in cooking, etc.). However, here are some of the most **iconic, award-winning, and beloved French cheeses**, each celebrated for its unique qualities:

### **Top Contenders for "Best" French Cheese**
1. **Comté (AOP)** – *The King of Alpine Cheeses*
   - **Why?** Aged 4 to 36+ months, Comté offers a perfect balance of nutty, caramelized, and fruity flavors. It’s versatile (great for melting, snacking, or grating) and consistently excellent.
   - **Best for:** Fondue, gratins, charcuterie boards.
   - **Fun fact:** Each wheel is unique due to variations in terroir and aging.

2. **Roquefort (AOP)** – *The Legendary Blue*
   - **Why?** Made from **raw sheep’s milk** and aged in the natural caves of Roquefort-sur-Soulzon, it’s creamy, tangy, and intensely flavorful with a perfect salt-crystal crunch.
   - **Best for:** Salads (like with 

SDKError: API error occurred: Status 429
{"object":"error","message":"Service tier capacity exceeded for this model.","type":"service_tier_capacity_exceeded","param":null,"code":"3505"}